In [ ]:
from catboost import CatBoostClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import recall_score, classification_report
import pandas as pd

# 기본 모델 정의
cat_model = CatBoostClassifier(
    random_state=42,
    verbose=0
)

# GridSearchCV 파라미터 설정 (꼭 너무 넓게 하지 말 것!)
param_grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [100, 300],
    'class_weights': [{0:1, 1:3}, {0:1, 1:5}, {0:1, 1:10}]
}

# GridSearchCV recall 기준
grid_search = GridSearchCV(
    estimator=cat_model,
    param_grid=param_grid,
    scoring='recall',
    cv=3,
    n_jobs=-1
)

# 학습 수행
grid_search.fit(X_train, y_train)

# 최적 모델
best_cat = grid_search.best_estimator_
y_pred = best_cat.predict(X_test)

# 결과 출력
print("✅ Best Params:", grid_search.best_params_)
print("🎯 Recall Score:", recall_score(y_test, y_pred))
print("\n📋 Classification Report:\n", classification_report(y_test, y_pred, target_names=["비이탈", "이탈"]))


In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# 혼동 행렬 시각화
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("예측값")
plt.ylabel("실제값")
plt.title("혼동 행렬 (Confusion Matrix)")
plt.tight_layout()
plt.show

In [ ]:
import joblib

# 저장할 데이터 구조 (CatBoost용으로 수정)
save_dict = {
    'model': best_cat,  # 👉 GridSearch로 얻은 최적 CatBoost 모델
    'encoders': label_encoders,  # 라벨 인코더 딕셔너리
    'features': X_train.columns,  # 사용된 피처 목록
    'metrics': {
        'model_name': 'CatBoost',
        'accuracy': 0.623,         # 여기에 실제 평가 결과 넣기
        'f1_score': 0.514,         # 실제 f1-score
        'recall': 0.932            # 높은 recall 반영
    }
}

# 저장
joblib.dump(save_dict, 'cb_model.joblib')
print("✅ cb_model.joblib 저장 완료")
